In [1]:
# Import necessory packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.mlab as mlab
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import warnings
import logging
warnings.filterwarnings('ignore')
logging.getLogger().setLevel(logging.ERROR)

In [2]:
# Importing the data files
sales = pd.read_csv('C:\\Users\\RadmehrFarzad\\OneDrive - Steampunk\\Desktop\\LEK\\all_sales.csv')
product = pd.read_csv('C:\\Users\\RadmehrFarzad\\OneDrive - Steampunk\\Desktop\\LEK\\product_hierarchy.csv')
store = pd.read_csv('C:\\Users\\RadmehrFarzad\\OneDrive - Steampunk\\Desktop\\LEK\\store_cities.csv')

#### Q6

Client wants to know what the main drivers for successful stores (success is defined as having high average monthly revenue). 

In [3]:
# Lasso Technique for Feature Selection
#X = df.drop(['revenue','product_id','store_id'], axis=1).values
#y = df['revenue'].values
#from sklearn.linear_model import Lasso
#names = df.drop('revenue',axis=1).columns
#lasso = Lasso(alpha=0.1)
#lasso_coef = lasso.fit(X, y).coef_

#### a

Build a prediction model to find the main drivers of successful stores. Prepare to explain these drivers to the client’s C-Suite.

In [4]:
sales['revenue'] = pd.to_numeric(sales['revenue'].str.replace("$","")).fillna(0)

In [5]:
df = pd.DataFrame(sales.groupby('store_id').agg({'price':['mean'],'revenue':['sum']})).reset_index()

In [6]:
df.columns = ['store_id','avg_price','total_revenue']

In [7]:
df = df.join(store.set_index('store_id'), on='store_id')

In [8]:
df['store_id'] = df['store_id'].astype(str).str.replace("S","").astype(int)
df['storetype_id'] = df['storetype_id'].astype(str).str.replace("ST","").astype(int)
df['city_id'] = df['city_id'].astype(str).str.replace("C","").astype(int)

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Define which columns should be encoded vs scaled
columns_to_encode = ['storetype_id']
columns_to_scale  = ['avg_price', 'total_revenue','store_size','store_radius_hh_income','store_radius_pop_density']

# Instantiate encoder/scaler
scaler = StandardScaler()
ohe    = OneHotEncoder(sparse=False)

# Scale and Encode Separate Columns
scaled_columns  = scaler.fit_transform(df[columns_to_scale]) 
encoded_columns =    ohe.fit_transform(df[columns_to_encode])

# Concatenate (Column-Bind) Processed Columns Back Together
processed_data = np.concatenate([scaled_columns, encoded_columns], axis=1)
processed_data = pd.DataFrame(processed_data)
processed_data.columns = ['avg_price','total_revenue','store_size','store_radius_hh_income','store_radius_pop_density','storetype_1','storetype_2','storetype_3','storetype_4']

In [10]:
processed_data

,avg_price,total_revenue,store_size,store_radius_hh_income,store_radius_pop_density,storetype_1,storetype_2,storetype_3,storetype_4
0,1.507394,1.878939,-1.372316,0.209294,2.316534,0.0,0.0,0.0,1.0
1,1.116372,0.745862,-1.301283,1.105493,1.152003,1.0,0.0,0.0,0.0
2,-0.819843,-0.416284,0.332473,0.098543,-0.555397,0.0,0.0,1.0,0.0
3,-0.477940,-0.281526,0.971770,-1.054602,-0.362869,0.0,0.0,1.0,0.0
4,0.587073,-0.493512,0.261441,-1.193402,0.139593,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
139,-0.965242,-0.571019,1.042802,-1.149390,-0.802886,0.0,0.0,1.0,0.0
140,-0.677866,-0.687374,-0.022691,1.381559,-1.065022,0.0,0.0,1.0,0.0
141,0.368211,-0.179543,0.119375,-0.863313,-0.199729,0.0,0.0,1.0,0.0
142,-0.887248,-0.656696,1.255901,2.598467,-0.993531,0.0,0.0,1.0,0.0


#### b

How would you convince the client that your model performs well (or not)?

by looking at the RMSE of train and test (error values). and also the errors for train and test are so close that we don't have overfitting or underfitting problem)

In [11]:
# Lasso Technique for Feature Selection
X = processed_data.drop('total_revenue', axis=1).values
y = processed_data['total_revenue'].values
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
reg_all = linear_model.LinearRegression()
reg_all.fit(X_train, y_train)

print('Root Mean Squared Error-Train:', np.sqrt(metrics.mean_squared_error(y_train, reg_all.predict(X_train))))
print('Root Mean Squared Error-Test:', np.sqrt(metrics.mean_squared_error(y_test, reg_all.predict(X_test))))


Root Mean Squared Error-Train: 0.33043670373410217
Root Mean Squared Error-Test: 0.8959999672809335


#### c

Client wants to open new stores to expand their footprint and would like to learn your recommendation whether to open the following potential stores.

In [12]:
d = {'avg_price': [10.5, 8.7,7.9], 'store_size': [62, 40,50], 'store_radius_hh_income':[52600,69800,78400],'store_radius_pop_density':[120.2,520.6,303.4],'storetype_1':[0,0,1],'storetype_1':[0,0,1],'storetype_2':[0,0,0],'storetype_3':[1,0,0],'storetype_4':[0,1,0]}

df1 = pd.DataFrame(data=d)

df1

,avg_price,store_size,store_radius_hh_income,store_radius_pop_density,storetype_1,storetype_2,storetype_3,storetype_4
0,10.5,62,52600,120.2,0,0,1,0
1,8.7,40,69800,520.6,0,0,0,1
2,7.9,50,78400,303.4,1,0,0,0


In [13]:
columns_to_scale1  = ['avg_price','store_size','store_radius_hh_income','store_radius_pop_density']
scaled_columns1  = scaler.fit_transform(df1[columns_to_scale1]) 
#columns_to_encode1 = ['storetype_id']
#encoded_columns1 =    ohe.fit_transform(df1[columns_to_encode1])
scaled_columns1 

array([[ 1.34890655,  1.26012384, -1.33630621, -1.18865082],
       [-0.30656967, -1.18599891,  0.26726124,  1.25790052],
       [-1.04233688, -0.07412493,  1.06904497, -0.0692497 ]])

In [14]:
#encoded_columns1

In [15]:
processed_data = pd.DataFrame(scaled_columns1)
#processed_data.columns = ['avg_price','store_size','store_radius_hh_income','store_radius_pop_density','storetype_1','storetype_2','storetype_3','storetype_4']
processed_data[['storetype_1','storetype_2','storetype_3','storetype_4']] = df1[['storetype_1','storetype_2','storetype_3','storetype_4']]
processed_data

,0,1,2,3,storetype_1,storetype_2,storetype_3,storetype_4
0,1.348907,1.260124,-1.336306,-1.188651,0,0,1,0
1,-0.306570,-1.185999,0.267261,1.257901,0,0,0,1
2,-1.042337,-0.074125,1.069045,-0.069250,1,0,0,0


In [16]:
reg_all.predict(processed_data)

array([-0.91456088,  0.8283431 , -0.06706555])

#### Candidate # 2 is the winner.